In [277]:
import numpy as np
import pandas as pd


In [278]:
data_path="/home/apoorva/Documents/kaggle/movie_reviews/train.tsv"

In [279]:
train = pd.read_csv(data_path, header=0, \
                    delimiter="\t", quoting=3)


In [280]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [281]:
train.shape


(156060, 4)

In [344]:
trainfull=train['Phrase'].values
trainfull.reshape(156060,1)
print(trainfull[0])
raw_train=trainfull[1:78030]
print (raw_train.shape)
test_phrase=trainfull[78030:]
print(test_phrase)





A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .
(78029,)
[ 'to watch Huppert scheming , with her small , intelligent eyes as steady as any noir villain , and to enjoy the perfectly pitched web of tension that Chabrol spins'
 'to watch Huppert scheming , with her small , intelligent eyes as steady as any noir villain , and'
 'to watch Huppert scheming , with her small , intelligent eyes as steady as any noir villain ,'
 ..., 'avuncular chortles' 'avuncular' 'chortles']


In [283]:
Sentiment_trainfull=train['Sentiment'].values
Sentiment_train=Sentiment_trainfull[1:78030]
Sentiment_test=Sentiment_trainfull[78030:]



In [284]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [285]:
stop_words = set(stopwords.words('english'))


In [286]:
def clean_words(raw_phrase):
    raw_phrase_notag=BeautifulSoup(raw_phrase,'lxml').get_text()
    raw_phrase_letter=re.sub("[^a-zA-Z]"," ",raw_phrase_notag)
    raw_phrase_nocaps=raw_phrase_letter.lower()
    raw_phrase_tokens=word_tokenize(raw_phrase_nocaps)
    raw_phrase_nostopword=[word for word in raw_phrase_tokens if word not in stop_words]
         
    return (" ".join(raw_phrase_nostopword))
def count_seqlen(phrase):
    phrase_tokens=word_tokenize(phrase)
    return (len(phrase_tokens))
    

In [287]:
clean_train=[]
seqlen=[]
test_seqlen=[]
size=raw_train.size
for i in range(0,size):
    
    clean_train.append(clean_words(raw_train[i]))
    seqlen.append(count_seqlen(clean_train[i]))
    test_seqlen.append(count_seqlen(test_phrase[i]))

/home/apoorva/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [288]:
   
print (clean_train[6767])   
print (seqlen[6767])

mesmerizing cinematic poem
3


In [312]:


max_seqlen=np.amax(test_seqlen)
print (max_seqlen)

52


In [313]:
wordsList = np.load('wordsList.npy')

wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')

In [314]:
def create_id_matrix(clean_train,size,max_seqlen):
    id_matrix=np.zeros((size,max_seqlen),dtype='int32')
    for phrase_count in range(0,size):
        tokens=word_tokenize(clean_train[phrase_count])
        word_count=0
        for word in tokens:
            try:
                id_matrix[phrase_count][word_count]=wordsList.index(word)
            except ValueError:
                id_matrix[phrase_count][word_count] = 399999
            word_count=word_count+1
    return (id_matrix)
            

In [315]:
size_test=test_phrase.size


In [316]:
id_matrix=create_id_matrix(clean_train,size,max_seqlen)


In [345]:
id_matrix_test=create_id_matrix(test_phrase,size_test,max_seqlen)


In [341]:
print(id_matrix)

[[  281 52287 11910 ...,     0     0     0]
 [  281     0     0 ...,     0     0     0]
 [    0     0     0 ...,     0     0     0]
 ..., 
 [  575  1716 72148 ...,     0     0     0]
 [  575  1716 72148 ...,     0     0     0]
 [  575  1716 72148 ...,     0     0     0]]


In [346]:
print (id_matrix_test.shape)

(78030, 52)


In [318]:
np.save('idsMatrix', id_matrix)
np.save('idsMatrix', id_matrix_test)

In [296]:
id_matrix[6767][1]

17603

In [319]:
id_matrix_train=id_matrix[1:78030]
id_matrix_test=id_matrix[78030:]

In [334]:
batchSize = 10
lstmUnits = 64
numClasses = 5
iterations = 100
max_seqlen=np.amax(test_seqlen)

In [321]:
import tensorflow as tf

tf.reset_default_graph()

In [322]:
labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, max_seqlen_test])

In [323]:
print (wordVectors.shape)
len(wordsList)

(400000, 50)


400000

In [324]:
numDimensions=50

In [325]:
data = tf.Variable(tf.zeros([batchSize, max_seqlen, numDimensions]))

data = tf.nn.embedding_lookup(wordVectors,input_data)

In [326]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)

lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)

value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

In [327]:
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

In [328]:
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [329]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [330]:
def getTrainBatch(counter):
    labels=np.zeros([batchSize,numClasses])
    arr = np.zeros([batchSize, max_seqlen])
    for i in range(counter,(counter+batchSize)):
        arr[i] = id_matrix_train[i]
        value=Sentiment_train[i]
        labels[i][value-1]=1
        counter=counter+1
    return arr, labels,counter


In [331]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(iterations):
    #Next Batch of reviews
    counter=0
    nextBatch, nextBatchLabels,counter = getTrainBatch(counter);
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
    print(i)
   
   

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000

...

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000

In [347]:
def getTestBatch(counter):
    labels=np.zeros([batchSize,numClasses])
    arr = np.zeros([batchSize, max_seqlen])
    for i in range(counter,(counter+batchSize)):
        arr[i] = id_matrix_test[i]
        value=Sentiment_test[i]
        labels[i][value-1]=1
        counter=counter+1
    return arr, labels,counter

In [348]:
iterations = 10
for i in range(iterations):
    counter=0
    nextBatch, nextBatchLabels,counter = getTestBatch(counter);
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)

Accuracy for this batch: 69.9999988079
Accuracy for this batch: 69.9999988079
Accuracy for this batch: 69.9999988079
Accuracy for this batch: 69.9999988079
Accuracy for this batch: 69.9999988079
Accuracy for this batch: 69.9999988079
Accuracy for this batch: 69.9999988079
Accuracy for this batch: 69.9999988079
Accuracy for this batch: 69.9999988079
Accuracy for this batch: 69.9999988079
